In [1]:
# import relevant libraries
import numpy as np
import pandas as pd
import gzip
import os
import glob
import re
import requests
from datetime import datetime
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import ast

## 1 : Extracting the logs

In [2]:
# get logs for month of February
folder_path = r'../../../lhstdata1/students/Gallica_logs/1LogGallicaFevrier2016'
all_files = glob.glob(folder_path + "/*.log.gz")
all_files.sort(key=lambda f: int(re.sub('\D', '', f)))

In [84]:
# take only part of the data, to make the size manageable
files = all_files[0:100]
# empty dataframe to store concatenation
dfs = []
for f in files:
    try:
        # read csv file into df
        df = pd.read_csv(f, encoding='UTF-8', sep ='\t', engine='python', header=None)
        # append to list of dfs
        dfs.append(df)
    except Exception as e:
        print(f"Error loading file {f}: {e}")

# from the list, concatenate the dataframes into a single one
combined_df = pd.concat(dfs, ignore_index=True)

In [85]:
combined_df.shape

(14739905, 1)

## 2 : Extracting features from logs 

Structure of a line : '##' then IP address, '##' then country (or null), '##' then city (or null), '##--' then date, then HTTP request in quotes, with the protocol number, followed by number of response (200 = OK), then size, then referrer website (or '-' if unknown)

In [86]:
# drop potential duplicate lines
combined_df = combined_df.drop_duplicates()

In [87]:
# get new shape
combined_df.shape

(14739810, 1)

In [88]:
# split the lines to recover meaningful information - ip address, country, city, date and request
logs_df = combined_df[0].str.split('##', expand=True)

In [89]:
# rename the columns with informative names
logs_df = logs_df.rename(columns = {1:"IPaddress", 2:"Country", 3:"City", 4:"Full_request"})

In [90]:
logs_df.head()

,0,IPaddress,Country,City,Full_request
0,,3c2af9233e11938ca3f73eb650d4af40,France,Nice,"- - [27/Feb/2016:18:01:04 +0100] ""GET /ark:/12..."
1,,c6bd521083f4402a71e65c33baa00f3e,Germany,Munich,"- - [27/Feb/2016:18:01:05 +0100] ""GET /ark:/12..."
2,,c6bd521083f4402a71e65c33baa00f3e,Germany,Munich,"- - [27/Feb/2016:18:01:05 +0100] ""GET /assets/..."
3,,c6bd521083f4402a71e65c33baa00f3e,Germany,Munich,"- - [27/Feb/2016:18:01:05 +0100] ""GET /assets/..."
4,,c6bd521083f4402a71e65c33baa00f3e,Germany,Munich,"- - [27/Feb/2016:18:01:05 +0100] ""GET /assets/..."


In [91]:
# drop columns that we don't use - Country, City
logs_df = logs_df.drop(['Country', 'City'], axis=1)

In [92]:
# extract dates and turn into Datetime
logs_df['Date'] = pd.to_datetime(logs_df.apply(lambda x: x['Full_request'].split("]")[0].split("[")[1], axis=1), format='%d/%b/%Y:%H:%M:%S %z')

In [95]:
# extract Request and Referrer fields
logs_df = logs_df.assign(
    Request=lambda x: x['Full_request'].str.split("\"").str[1].str.split().str[:2].str.join(' '),
    Referrer=lambda x: x['Full_request'].str.split("\"").str[3]
)

In [96]:
# drop full request column 
logs_df = logs_df.drop(['Full_request'], axis=1)

In [97]:
# function to extract ark from the request
def extract_ark(request):
    # check if '12148' is in the request (since it is specific to gallica ARKs)
    if '12148' not in request:
        return '-'
    
    ark = '-'
    
    # regular expression pattern, 12148 is specific to gallica
    pattern = r'/12148/([^/.]+)'
    
    # use regex to find ark in request string
    match = re.search(pattern, request)
    if match:
        ark = match.group(1)
    
    return ark

In [98]:
# extract ark
logs_df['Ark'] = logs_df.apply(lambda x: extract_ark(x['Request']), axis=1)

In [99]:
logs_df.head(10)

,0,IPaddress,Date,Request,Referrer,Ark
0,,3c2af9233e11938ca3f73eb650d4af40,2016-02-27 18:01:04+01:00,GET /ark:/12148/bpt6k57843235,http://www.google.fr/url?sa=t&rct=j&q=&esrc=s&...,bpt6k57843235
1,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /ark:/12148/btv1b8490545v,http://images.google.de/imgres?imgurl=http://g...,btv1b8490545v
2,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/vendor/bootstra...,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-
3,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/main.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-
4,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/persoScrollBar.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-
5,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/panes.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-
6,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/visualiseur.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-
7,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/thetiere/fonts/...,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-
8,,43fdf9ff858eb2aa045661bf488124df,2016-02-27 18:01:05+01:00,GET /ark:/12148/btv1b69277000/f1.highres,https://www.google.fr/,btv1b69277000
9,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:06+01:00,GET /assets/static/stylesheets/fonts/pictos.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-


In [100]:
# helper functions
def contains_number_or_special_chars(word):
    special_chars = '/()'
    return any(char.isdigit() for char in word) or any(char in special_chars for char in word)

def remove_term(term):
    words = term.split()
    return all(contains_number_or_special_chars(word) for word in words)

In [101]:
# function to keep only clean search terms
def clean_search_terms(terms):
    return [term for term in terms if not remove_term(term)]

In [102]:
# function to extract search terms from the request

def extract_search_terms(request):
    # check if 'search' is in the request
    if 'search' not in request:
        return []
    
    # parse URL
    parsed_url = urlparse(request)
    # extract query parameters
    query_params = parse_qs(parsed_url.query)
    # extract search query from query parameters
    search_query = query_params.get('query', [''])[0]
    # URL-decode the search query
    search_query = unquote(search_query)
    # extract search terms using regular expression
    search_terms = re.findall(r'"([^"]+)"', search_query)
    
    return clean_search_terms(search_terms)

In [103]:
# extract and clean search terms
logs_df['search_terms'] = logs_df.apply(lambda x: extract_search_terms(x['Request']), axis=1)

In [104]:
logs_df[:20]

,0,IPaddress,Date,Request,Referrer,Ark,search_terms
0,,3c2af9233e11938ca3f73eb650d4af40,2016-02-27 18:01:04+01:00,GET /ark:/12148/bpt6k57843235,http://www.google.fr/url?sa=t&rct=j&q=&esrc=s&...,bpt6k57843235,[]
1,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /ark:/12148/btv1b8490545v,http://images.google.de/imgres?imgurl=http://g...,btv1b8490545v,[]
2,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/vendor/bootstra...,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]
3,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/main.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]
4,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/persoScrollBar.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]
5,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/panes.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]
6,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/visualiseur.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]
7,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/thetiere/fonts/...,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]
8,,43fdf9ff858eb2aa045661bf488124df,2016-02-27 18:01:05+01:00,GET /ark:/12148/btv1b69277000/f1.highres,https://www.google.fr/,btv1b69277000,[]
9,,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:06+01:00,GET /assets/static/stylesheets/fonts/pictos.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,-,[]


## 3 : Enriching log data

We want to enrich the data by adding additional information about the requested document (if there was one) : Dewey class, type of document, visibility. Among other things, this will help create a diversity metric for the sessions later on. To examine the visibility of a document, we want to know : how many times was it seen over all sessions ? For that we create a new dataframe with unique arks associated with the count of the number of times they were seen in this dataframe. Creating a new dataframe and storing it means we can later concatenate them if there are many. We also assume that one person increments the visibility of an ark once. So even if there are ten logs with the same ark, if it is one IP address, we count it as 1 count.

In [118]:
# replace unknown arks by None
logs_df.loc[logs_df['Ark'] == '-', 'Ark'] = None
# keep rows with not None arks
filtered_arks = logs_df.dropna(subset=['Ark'])
# drop duplicates based on both ark and ip address
unique_ip_arks = filtered_arks.drop_duplicates(subset=['Ark', 'IPaddress'])

In [119]:
len(unique_ip_arks)

1614176

In [120]:
# group by ark and count occurrences
ark_counts = unique_ip_arks.groupby('Ark').size().reset_index(name='Count')

In [121]:
ark_counts.shape

(641529, 2)

In [122]:
# save unique arks with their count
ark_counts.to_csv("data_temp_month/unique_arks/unique_arks_counts0.csv", index=False)

In [109]:
# save the clean logs
logs_df.to_csv("data_temp_month/clean_logs0.csv", index=False)

We save the clean logs and the arks and their counts. The next step will be request them, in the next notebook. 

In [30]:
# seperate empty and non empty arks
non_empty_arks_df = arks_df[pd.notnull(arks_df['Type'])]
# find empty typedoc and create new dataframe to request these arks
empty_arks_df = arks_df[pd.isnull(arks_df['Type'])]

In [31]:
nb_empty_arks = len(empty_arks_df)
print("There are",nb_empty_arks , "empty arks, which is", nb_empty_arks/len(arks_df)*100, "% of the arks.")

There are 19317 empty arks, which is 4.041289480345864 % of the arks.


In [32]:
# save empty and non empty arks for now
non_empty_arks_df.to_csv("arks_final/arks_non_empty.csv")
empty_arks_df.to_csv("arks_final/arks_empty.csv")

In [34]:
# seems some of them are not cut at the right place at end, we fix that then re-request
empty_arks_df = empty_arks_df.copy()
# Apply cleaning function to 'Ark' column
empty_arks_df.loc[:, 'Ark_clean'] = empty_arks_df['Ark'].apply(clean_ark)

In [35]:
empty_arks_df

,Ark,Theme,Type,Ark_clean
458674,btv1b7720377q,NaN,NaN,btv1b7720377q
458675,bpt6k5795138h,NaN,NaN,bpt6k5795138h
458676,bpt6k298770b,NaN,NaN,bpt6k298770b
458677,btv1b5966539r,NaN,NaN,btv1b5966539r
458678,bpt6k5489230s,NaN,NaN,bpt6k5489230s
...,...,...,...,...
477986,bpt6k815278%20,NaN,NaN,bpt6k815278
477987,bpt6k6451820c%20,NaN,NaN,bpt6k6451820c
477988,bpt6k6464419j%20,NaN,NaN,bpt6k6464419j
477989,bpt6k62515285%20,NaN,NaN,bpt6k62515285


In [36]:
empty_arks_df.to_csv("arks_temp/arks_2ndrun.csv")

Some of the clean arks values are not arks. We re-clean with a function that specifies that arks must start with a 'b'.

In [37]:
b_mask = empty_arks_df['Ark_clean'].str.startswith('b', na=False)
filtered_empty_arks = empty_arks_df[b_mask]

In [38]:
filtered_empty_arks

,Ark,Theme,Type,Ark_clean
458674,btv1b7720377q,NaN,NaN,btv1b7720377q
458675,bpt6k5795138h,NaN,NaN,bpt6k5795138h
458676,bpt6k298770b,NaN,NaN,bpt6k298770b
458677,btv1b5966539r,NaN,NaN,btv1b5966539r
458678,bpt6k5489230s,NaN,NaN,bpt6k5489230s
...,...,...,...,...
477986,bpt6k815278%20,NaN,NaN,bpt6k815278
477987,bpt6k6451820c%20,NaN,NaN,bpt6k6451820c
477988,bpt6k6464419j%20,NaN,NaN,bpt6k6464419j
477989,bpt6k62515285%20,NaN,NaN,bpt6k62515285


Some arks have the structure btv1b90039### and when requested, yield "Erreur d'utilisation.500". We assume these refer to documents that are not available anymore on gallica. We re-request the missing arks to reduce the percentage of missing values.

In [39]:
# we also know some documents are not available anymore so the request won't work, some ark values will not have metadata
arks_run2_df = pd.DataFrame()
clean_arks_run2_df = apply_with_progress(filtered_empty_arks, lambda row: extract_metadata(row['Ark_clean']), arks_run2_df, 10000, 'arks_temp/fill_run2')

# save the remaining results to CSV
clean_arks_run2_df.to_csv('arks_final/filling_arks.csv', index=False)

  0%|▏                                     | 84/19194 [25:48<6:26:29,  1.21s/it]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


  0%|▏                                     | 88/19194 [25:49<2:55:01,  1.82it/s]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


  0%|▏                                     | 92/19194 [25:49<1:31:43,  3.47it/s]

...

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|██████████████████████████████████▉| 19191/19194 [3:46:00<00:00, 15.16it/s]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|███████████████████████████████████| 19194/19194 [3:46:00<00:00,  1.42it/s]


Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


In [40]:
# now concatenate the obtained arks with the rest
arks_final_df = pd.concat([non_empty_arks_df, clean_arks_run2_df], ignore_index=True)

In [41]:
arks_final_df

,Ark,Theme,Type
0,btv1b8459670z,NaN,carte
1,btv1b59624343,NaN,carte
2,btv1b53036648q,NaN,carte
3,btv1b530395328,NaN,carte
4,btv1b5962468w,NaN,carte
...,...,...,...
477863,bpt6k815278,,
477864,bpt6k6451820c,,
477865,bpt6k6464419j,,
477866,bpt6k62515285,,


In [42]:
# drop potential duplicates, save final complete arks
arks_final_df = arks_final_df.drop_duplicates(subset=['Ark'])
arks_final_df.to_csv("data/arks_final.csv")

In [48]:
arks_final_df.loc[arks_final_df['Type'] == '', 'Type'] = np.nan

In [49]:
# get empty arks after 2nd run
empty_arks_2_df = arks_final_df[pd.isna(arks_final_df['Type'])]

In [50]:
nb_empty_arks = len(empty_arks_2_df)
print("There are now", nb_empty_arks , "empty arks, which is", nb_empty_arks/len(arks_final_df)*100, "% of the arks.")

There are now 17733 empty arks, which is 3.716675050301811 % of the arks.


In [51]:
# save the new empty and all arks dataframes
empty_arks_2_df.to_csv("arks_final/arks_empty.csv")
arks_final_df.to_csv("arks_final/processed_arks.csv")

## 5: Concatenating the pre-processed data

In [52]:
# merge DataFrames based on the 'Ark' column, keeping all rows from 'logs_df'
logs_arks_df = pd.merge(logs_df, arks_final_df, on='Ark', how='left')

In [53]:
logs_arks_df.columns

Index([             0,    'IPaddress',      'Country',         'City',
               'Date',      'Request',     'Referrer',          'Ark',
       'search_terms',   'visibility',        'Theme',         'Type'],
      dtype='object')

In [55]:
# check length is the same as the beggining
len(logs_arks_df)

10042040

#### Cleaning Nans in 'Theme' and 'Type' fields

The theme field can be Nan if either the document has no Dewey class, or if the document metadata was not retrieved. To differenciate these cases, we put Theme = 'No_dewey_class' when the document has no Dewey class, and 'Unknown' for Theme and Type when the document metadata was not available.
The Type and Theme can also be Nan if the log has no ark (for example, when an action is done). In this case, we give it the value 'No_ark'.

In [56]:
known_type_mask = logs_arks_df['Type'].notna() & logs_arks_df['Theme'].isna()
# replacing 'Theme' values with -1 in rows where 'Theme' is Nan and 'Type' is known, indicating a document with no Dewey class
logs_arks_df.loc[known_type_mask, 'Theme'] = 'No_dewey_class'

In [57]:
unknown_ark_mask = logs_arks_df['Type'].isna() & logs_arks_df['Theme'].isna()
# replacing 'Theme' and 'Type' values with 'Unknown, indicating the metadata was not available
logs_arks_df.loc[unknown_ark_mask, ['Theme', 'Type']] = 'Unknown', 'Unknown'

In [60]:
logs_arks_df.loc[logs_arks_df['Ark'] == '', 'Ark'] = np.nan
no_ark_type_mask = logs_arks_df['Ark'].isna()
# replacing 'Theme' and 'Type' values with 'no_ark' where 'Ark' is NaN
logs_arks_df.loc[no_ark_type_mask, ['Theme', 'Type']] = 'No_ark', 'No_ark'

In [61]:
logs_arks_df

,0,IPaddress,Country,City,Date,Request,Referrer,Ark,search_terms,visibility,Theme,Type
0,,e7fdec50f50253f6796d61b5382155f8,null,null,31/Jan/2016:18:59:19 +0100,GET /ark:/12148/bpt6k70211m,-,bpt6k70211m,[],9.643857e-03,84.0,monographie
1,,5d1edaaaa9cf4772ce828127ce3e523a,United States,Chapel Hill,31/Jan/2016:18:59:20 +0100,GET /ark:/12148/bpt6k622723.planchecontact.r=e...,-,bpt6k622723,[],4.979068e-07,70.0,monographie
2,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,31/Jan/2016:18:59:21 +0100,GET /services/ajax/extract/ark:/12148/bpt6k866...,http://gallica.bnf.fr/services/engine/search/s...,bpt6k866717,[],7.369021e-06,1.0,fascicule
3,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,31/Jan/2016:18:59:21 +0100,GET /services/engine/search/sru?operation=sear...,http://gallica.bnf.fr/services/engine/search/s...,NaN,"[cb34348270x_date, 1830, Renduel]",3.690648e-01,No_ark,No_ark
4,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,31/Jan/2016:18:59:21 +0100,GET /services/ajax/extract/ark:/12148/bpt6k653...,http://gallica.bnf.fr/services/engine/search/s...,bpt6k65309826,[],6.970695e-07,1.0,fascicule
...,...,...,...,...,...,...,...,...,...,...,...,...
10042035,,be5b870a4a00cff5338979f7539203cc,United States,Menlo Park,02/Feb/2016:05:30:18 +0100,GET /ark:/12148/btv1b85565468.thumbnail,-,btv1b85565468,[],7.966509e-07,No_dewey_class,image
10042036,,be5b870a4a00cff5338979f7539203cc,United States,Menlo Park,02/Feb/2016:05:30:19 +0100,GET /assets/static/images/entete/facebook.png,-,NaN,[],3.690648e-01,No_ark,No_ark
10042037,,489ea6e450f8bc90a93565e9f9be2e63,United States,San Francisco,02/Feb/2016:05:30:24 +0100,GET /ark:/12148/bpt6k105074g/f33.highres,-,bpt6k105074g,[],8.962322e-07,60.0,monographie
10042038,,489ea6e450f8bc90a93565e9f9be2e63,United States,San Francisco,02/Feb/2016:05:30:24 +0100,GET /ark:/12148/bpt6k105074g/f243.highres,-,bpt6k105074g,[],8.962322e-07,60.0,monographie


## 6 : Final step : saving the enriched logs

In [62]:
logs_arks_df.to_csv("data/enriched_logs.csv")

Our data is now pre-processed and enriched. The next step will be to create user sessions from it, and then classify part of these sessions as Rabbit Holes.